# Word Embedding

## 1. Natural Language Processing
- 컴퓨터가 인간이 사용하는 언어를 이해하고 분석할 수 있게 하는 분야
- 그렇지만 컴퓨터가 단어에 대해 인지하기 위해서는 수치적으로 단어를 표현할 수 있어야 한다. 
- 이전까지는 n개의 단어 사전이 있었을 때 해당 단어에만 1을 넣고 나머지는 9을 넣는 **One hot encoding** 방법으로 단어를 표현했다. 하지만 이 방법은 컴퓨터가 단어의 본질적인 특징과 다른 단어간의 관련성을 이해하지 못한다는 단점을 지닌다.
- 이러한 단점을 극복하지 위해 단어 자체가 가지는 의미를 벡터화하는 방식을 고안했다. 이런 방식으로 단어를 표현하면 단어 간의 유사도를 측정할 수 있고 수치적으로도 쉽게 다를 수 있다. 예를들어, 일본이란 벡터에서 도쿄를 빼고 서울을 넣으면 한국이라는 결과를 얻는 식이다.


## 2. Word2Vec
- word2vec은 word embedding방법론 중 하나로 2013년 구글에서 발표된 연구이며 여러 연구자들이 모여 만든 continuous word embedding 학습 모형이다.
- 이 연구에서는 학습을 시키기 위한 네트워크 모델을 두가지 제시하였다.

    1) CBOW(Continuous Bag-of-Words) : source context에서 target을 예측한다. 
        ex) 'the cat sits on the' 라는 context에서 'rug' 라는 타겟 단어를 예측한다.
        
    2) Skip-gram 모델 : target으로부터 source context를 예측한다.
        ex) 'rug'로 부터 'the cats on the'라는 소스 컨텍스트를 예측한다.
   
    ### 1) CBOW 
    ![cbow](picture/CBOW.png)
    - cbow모델은 크게 input layer, projection layer, output layer로 이루어져 있다.
    - projection layer의 개수는 target을 얼마의 길이의 벡터로 표현해 줄지에 따라 다르다.
    - 이 모델은 이전 모델에 비해 계산량이 매우 줄엇다는 특징이 있다.
    ### 2) Skip-gram
    <br/>
    - cbow와의 반대방향의 모델이라 생각하면 된다. 현재 주어진 단어하나를 가지고 주위에 등장하는 나머지 몇가지의 단어들의 등장 여부를 유추하는 것이다.
    - "가까이 위치에 잇는 단어일수록 현재 단어와 관련이 더 많은 단어일 것"이라고 가정한다.
    - 이러한 가정에 따라 input단어가 주어졌을 때 그 단어 주위의 단어들을 무작위로 선택한다. 그러면 network는 모든 단어에 대하여 선택된 단어가 인접한 단어일 가능성(probability)을 구한다.
    - 만약 라면이라는 단어를 입력했다면 우동이라는 단어가 근처에 있을 확률은 학교나 의자보다 훨씬 높을 것이다.
    
<br/>[출처](https://shuuki4.wordpress.com/2016/01/27/word2vec-관련-이론-정리/)
,[학습방법](http://solarisailab.com/archives/374)
<br/>


### method
1. training data
    ![skip-gram-training](picture/skip-gram-training.png)
    - 우선 모든 단어들은 one-hot vector로 표현되어야 한다. 
    - 학습 문서 내의 input들은 window size 내의 모든 단어의 쌍으로 학습된다.
    <br/><br/>
2. 학습구조<br/>
    ![skip-gram](picture/skip-gram.png)
    - input vector는 지정된 hidden layer(projection layer)로 줄어들고, 이 과정에서 activation이 일어나지 않는다.
    - 이러한 hidden layer에서 다시 input vector과 동일한 길이의 output layer로 mapping된다.
    - 이 결과값이 softmax 함수를 이용해서 classification하는데 이용된다.
    - 아래의 식을 최대화시키는 방향으로 학습이 진행된다.
    $$ P(o|c)=\frac{exp(u_o^Tv_c)}{\sum_{w=1}^{W}exp(u_w^Tv_c)}$$
    <br/>
    - 분자 부분은 중심단어(c)에 해당하는 벡터와 주변단어(o)에 해당하는 벡터의 내적값을 높인다. (유사도를 높인다)
    - 분모는 윈도우 내에 등장하지 않는 단어는 중심단어와의 유사도를 낮춘다는 의미   
    <br/><br/>
3. 결과
    - classification에 사용되었던 output layer는 사용되지 않고 hidden layer의 vector값이 이용되게 된다. 
    <br/><br/>
4. 학습트릭

    1) **subsampling frequent words**
    - Word2Vec은 vector이 weight matrix가 커질수록 계산량이 커지고 이는 단어 수가 많아질수록 커진다.
    - 이에 따라 너무 자주 등장하는 is, the같은 단어는 확률적으로 줄이기로 했다. 이어따라 빈도가 높은 단어는 일정확률로 학습에 제외시키므로써 계산량을 감소시킨다.
    
    2) **negative sampling**
    - softmax를 활용해서 classification을 할려면 중심단어와 나머지 모든 단어에 대해 내적한뒤 exp을 취해줘야 한다. 따라서 이때 전체 단어를 대상으로 하지 않고 일부 단어만 뽑아서 계산을 한다.
    - 이렇게 target(중심)단어가 아닌 다른 단어들 중 일부만 뽑아서 softmax 함수를 계산하는 것이 negative sampling이다.